<a href="https://colab.research.google.com/github/antopeza/Final-Project-Brainster/blob/main/Doc2Vec_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from numpy import vstack
import matplotlib.pyplot as plt
import os 

import re

import string
from string import punctuation

from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize

import gensim
from gensim.parsing.preprocessing import remove_stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from imblearn.over_sampling import SMOTE
from collections import Counter

#from scikeras.wrappers import KerasClassifier

import pickle
from pickle import dump
from pickle import load

from imblearn.over_sampling import SMOTE
from collections import Counter

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [3]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
stops = set(stopwords.words('english'))
#print(stops)

In [5]:
stemmer =  PorterStemmer()
lemmer = WordNetLemmatizer()

In [6]:
#df_raw = pd.read_csv(path)
df_raw = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')

In [ ]:
# print('shape of df  = ', df_raw.shape)
# print()
# df_raw.head(2)

shape of df  =  (404290, 6)



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [ ]:
#df_raw[df_raw.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [ ]:
#df_raw = df_raw.fillna("")
#df_raw.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
# randomly selected samples from df for the new df to perform operations
df = df_raw.sample(5000)
df = df[['question1', 'question2', 'is_duplicate']]
print(df.shape)

(5000, 3)


In [ ]:
#df_copy = df.copy()

# Clean Data

In [8]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
    #sentence = re.sub(r'\s{2,}', ' ', sentence
   
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
    if stopwords:
        sentence = remove_stopwords(sentence)  
        
    if lemmatize:
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
        wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag is None:
                res_words.append(word)
            else :
                res_words.append(lemmer.lemmatize(word, tag))
        sentence = " ".join(res_words)
        
    if stem:
        sent_stemmed = ''
        for word in sentence.split():
            sent_stemmed += ' '+ stemmer.stem(word)
        sentence = sent_stemmed
   
    return sentence

def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
    #sents1 = dfq[['question1']]  
    #sents2 = dfq[['question2']]
    cleaned_sentences1 = []
    cleaned_sentences2 = []
    
    for index, row in df.iterrows():
        # print (index, row)
        cleaned1 = clean_sentence(row['question1'], stopwords, lemmatize, stem) 
        cleaned2 = clean_sentence(row['question2'], stopwords, lemmatize, stem)

        cleaned_sentences1.append(cleaned1)
        cleaned_sentences2.append(cleaned2)
    return cleaned_sentences1, cleaned_sentences2

In [9]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df_raw['is_duplicate']
    return X_temp

#### With Stopwords, lemmatized

In [10]:
# With stopwords lemmatized, not stemmed
q1_sw_lemma, q2_sw_lemma = get_cleaned_senteces(df, stopwords = False, lemmatize = True, stem = False)
assert len(q1_sw_lemma) == len(q2_sw_lemma)
df_sw_lemma = to_df(q1_sw_lemma, q2_sw_lemma, df)
df_sw_lemma.shape

(5000, 3)

#### With Stopwords, stemmed

In [11]:
q1_sw_stem, q2_sw_stem = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = True)
assert len(q1_sw_stem) == len(q2_sw_stem)
df_sw_stem = to_df(q1_sw_stem, q2_sw_stem, df)
df_sw_stem.shape

(5000, 3)

#### With stopwords, not lemmatized nor stemmed

In [12]:
q1_sw, q2_sw = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False)
assert len(q1_sw) == len(q2_sw)
df_sw = to_df(q1_sw, q2_sw, df)
df_sw.shape

(5000, 3)

#### Without Stopwords, lemmatized


In [13]:
q1_lemma, q2_lemma = get_cleaned_senteces(df, stopwords = True, lemmatize = True, stem = False)
assert len(q1_lemma) == len(q2_lemma)
df_lemma = to_df(q1_lemma, q2_lemma, df)
df_lemma.shape

(5000, 3)

#### Without Stopwords, stemmed

In [14]:
q1_stem, q2_stem = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = True)
assert len(q1_stem) == len(q2_stem)
df_stem = to_df(q1_stem, q2_stem, df)
df_stem.shape

(5000, 3)

#### Without Stopwords, not lemmatized nor stemmed

In [15]:
q1_out, q2_out = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = False)
assert len(q1_out) == len(q2_out)
df_out = to_df(q1_out, q2_out, df)
df_out.shape

(5000, 3)

In [16]:
display ('With stopwords, without lemma and stem', df_sw.iloc[[22]])
display ('With stopwords, lemmatized and without stem', df_sw_lemma.iloc[[22]])
display ('With stopwords, stemmed and without lemma', df_sw_stem.iloc[[22]])
display ('Without stopwords, lemma and stem', df_out.iloc[[22]])
display ('Without stopwords and stem, lemmatized', df_lemma.iloc[[22]])
display ('Without stopwords and lemma, stemmed', df_stem.iloc[[22]])

'With stopwords, without lemma and stem'

,q1,q2,is_duplicate
397754,how important should sex be in a relationship,how important is sex in a successful relationship,1


'With stopwords, lemmatized and without stem'

,q1,q2,is_duplicate
397754,how important should sex be in a relationship,how important be sex in a successful relationship,1


'With stopwords, stemmed and without lemma'

...

,q1,q2,is_duplicate
397754,important sex relationship,important sex successful relationship,1


'Without stopwords and stem, lemmatized'

,q1,q2,is_duplicate
397754,important sex relationship,important sex successful relationship,1


'Without stopwords and lemma, stemmed'

,q1,q2,is_duplicate
397754,import sex relationship,import sex success relationship,1


In [ ]:
# print('Value counts:\n', X_sw_train['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_train['is_duplicate'].value_counts()/X_sw_train['is_duplicate'].count()*100)
# X_sw_train['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of train labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# print('Value counts:\n', X_sw_test['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_test['is_duplicate'].value_counts()/X_sw_test['is_duplicate'].count()*100)
# X_sw_test['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of test labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# display (X_sw_train.head(2))
# display(X_sw_test.head(2))
# display(df_clean.head(2))

In [ ]:
#X_sw_test[X_sw_test.isnull().any(axis=1)]

### TOKENIZATION

In [17]:
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def token(text):
    text = text.apply(lambda x: tokenization(x))
    return text

In [18]:
#Tokenize - lemmatized data without stopwords
df_lemma['q1'] = df_lemma['q1'].pipe(token) 
df_lemma['q2'] = df_lemma['q2'].pipe(token) 
df_lemma.head(2)

,q1,q2,is_duplicate
84883,"[happen, carol, husband, mike, wife, brady, bu...","[happen, husbandwife, midlife, crisis]",0
6371,"[best, password, manager]","[whats, good, password, manager]",1


In [19]:
# Tokenize - lemmatized data with stopwords
df_sw_lemma['q1'] = df_sw_lemma['q1'].pipe(token) 
df_sw_lemma['q2'] = df_sw_lemma['q2'].pipe(token) 
df_sw_lemma.head(2)

,q1,q2,is_duplicate
84883,"[what, happen, to, carol, first, husband, and,...","[what, will, happen, to, your, husbandwife, af...",0
6371,"[what, be, the, best, password, manager]","[whats, a, good, password, manager]",1


In [20]:
# Tokenize - Stemmed data with stopwords
df_sw_stem['q1'] = df_sw_stem['q1'].pipe(token) 
df_sw_stem['q2'] = df_sw_stem['q2'].pipe(token) 
df_sw_stem.head(2)

,q1,q2,is_duplicate
84883,"[, what, happen, to, carol, first, husband, an...","[, what, will, happen, to, your, husbandwif, a...",0
6371,"[, what, is, the, best, password, manag]","[, what, a, good, password, manag]",1


In [21]:
# Tokenize - Stemmed data without stopwords
df_stem['q1'] = df_stem['q1'].pipe(token) 
df_stem['q2'] = df_stem['q2'].pipe(token) 
df_stem.head(2)

,q1,q2,is_duplicate
84883,"[, happen, carol, husband, mike, wife, bradi, ...","[, happen, husbandwif, midlif, crisi]",0
6371,"[, best, password, manag]","[, what, good, password, manag]",1


In [22]:
# Tokenize - With stopwords, not lemmatized nor stemmed
df_sw['q1'] = df_sw['q1'].pipe(token) 
df_sw['q2'] = df_sw['q2'].pipe(token) 
df_stem.head(2)

,q1,q2,is_duplicate
84883,"[, happen, carol, husband, mike, wife, bradi, ...","[, happen, husbandwif, midlif, crisi]",0
6371,"[, best, password, manag]","[, what, good, password, manag]",1


In [23]:
# Tokenize - Without stopwords, not lemmatized nor stemmed
df_out['q1'] = df_out['q1'].pipe(token) 
df_out['q2'] = df_out['q2'].pipe(token) 
df_out.head(2)

,q1,q2,is_duplicate
84883,"[happened, carols, husband, mikes, wife, brady...","[happen, husbandwife, midlife, crisis]",0
6371,"[best, password, manager]","[whats, good, password, manager]",1


In [24]:
df_sw_lemma.head(2)

,q1,q2,is_duplicate
84883,"[what, happen, to, carol, first, husband, and,...","[what, will, happen, to, your, husbandwife, af...",0
6371,"[what, be, the, best, password, manager]","[whats, a, good, password, manager]",1


# Doc2Vec function

In [ ]:
# Creating DataFrame for Evaluations
result_tf = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_tf.head()

In [ ]:
def res(model, acc, precision, recall, f1_score, log_loss):
    result_tf.loc[len(result_tf)]   = [model, acc, precision, recall, f1_score, log_loss]
    pd.set_option('display.max_rows', None)
    display (result_tf)
    return result_tf

In [25]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def doc2vec(df):
  q1 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q1'])]
  q2 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q2'])]
  return q1, q2

In [26]:
def model_doc2vec(q1, dm, vector, window, epochs):
  q1 = Doc2Vec(q1, dm = dm, vector_size = vector, window = window, min_count = 2, epochs = epochs)
  return q1

### Run Doc2Vec Function

In [27]:
#lemmatized, without stopwords
q1_lemma_d2v, q2_lemma_d2v = doc2vec(df_lemma)
assert len(q1_lemma_d2v) == len(q2_lemma_d2v)

In [28]:
#lemmatized with stopwords
q1_sw_lemma_d2v, q2_sw_lemma_d2v = doc2vec(df_sw_lemma)
assert len(q1_sw_lemma_d2v) == len(q2_sw_lemma_d2v)

In [29]:
#Stemmed without stopwords
q1_stem_d2v, q2_stem_d2v = doc2vec(df_stem)
assert len(q1_stem_d2v) == len(q2_stem_d2v)

In [30]:
#Stemmed with stopwords
q1_sw_stem_d2v, q2_sw_stem_d2v = doc2vec(df_sw_stem)
assert len(q1_sw_stem_d2v) == len(q2_sw_stem_d2v)

In [31]:
#With Stopwords, not lemmatized, not stemmed
q1_sw_d2v, q2_sw_d2v = doc2vec(df_sw)
assert len(q1_sw_d2v) == len(q2_sw_d2v)

In [32]:
#Without stopwords, not lemmatized, not stemmed 
q1_out_d2v, q2_out_d2v = doc2vec(df_out)
assert len(q1_out_d2v) == len(q2_out_d2v)

### Activating Doc2Vec model

In [33]:
#PARAMETER FOR DOC2VEC MODEL 
dm = 1
window = 3
vector = 100
epochs = 30

In [34]:
#lemmatized, without stopwords
X_q1_lemma_d2v = model_doc2vec(q1_lemma_d2v, dm, vector, window, epochs)
X_q2_lemma_d2v = model_doc2vec(q2_lemma_d2v, dm,  vector, window, epochs)

In [38]:
#lemmatized, with stopwords
X_q1_sw_lemma_d2v = model_doc2vec(q1_sw_lemma_d2v, dm, vector, window, epochs)
X_q2_sw_lemma_d2v = model_doc2vec(q2_sw_lemma_d2v, dm, vector, window, epochs)

In [37]:
#Stemmed, without stopwords
X_q1_stem_d2v = model_doc2vec(q1_stem_d2v, dm, vector, window, epochs)
X_q2_stem_d2v = model_doc2vec(q2_stem_d2v, dm, vector, window, epochs)

In [39]:
#Stemmed, with stopwords
X_q1_sw_stem_d2v = model_doc2vec(q1_sw_stem_d2v, dm, vector, window, epochs)
X_q2_sw_stem_d2v = model_doc2vec(q2_sw_stem_d2v, dm, vector, window, epochs)

In [40]:
#With Stopwords, not lemmatized, not stemmed
X_q1_sw_d2v = model_doc2vec(q1_sw_d2v, dm, vector, window, epochs)
X_q2_sw_d2v = model_doc2vec(q2_sw_d2v, dm, vector, window, epochs)

In [41]:
#Without stopwords, not lemmatized, not stemmed 
X_q1_out_d2v = model_doc2vec(q1_out_d2v, dm, vector, window, epochs)
X_q2_out_d2v = model_doc2vec(q2_out_d2v, dm, vector, window, epochs)


#### Convert to array 

In [42]:
# Lemmatized, without stopwords, not stemmed

size = len(df) - 1
arr_q1_lemma = X_q1_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_lemma_d2v[i+1]
  arr_q1_lemma = np.vstack([arr_q1_lemma, arrT])

arr_q2_lemma = X_q2_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_lemma_d2v[i+1]
  arr_q2_lemma = np.vstack([arr_q2_lemma, arrT])

assert arr_q1_lemma.shape == arr_q2_lemma.shape

#arr_q1_lemma = create_arrays(X_q1_lemma_d2v)
#arr_q2_lemma = create_arrays(X_q2_lemma_d2v)

In [43]:
# Lemmatized, with stopwords, not stemmed

size = len(df) - 1
arr_q1_sw_lemma = X_q1_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_lemma_d2v[i+1]
  arr_q1_sw_lemma = np.vstack([arr_q1_sw_lemma, arrT])

arr_q2_sw_lemma = X_q2_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_lemma_d2v[i+1]
  arr_q2_sw_lemma = np.vstack([arr_q2_sw_lemma, arrT])

assert arr_q1_sw_lemma.shape == arr_q2_sw_lemma.shape

# arr_q1_sw_lemma = create_arrays(X_q1_sw_lemma_d2v)
# arr_q2_sw_lemma = create_arrays(X_q2_sw_lemma_d2v) 

In [44]:
# Stemmed, without stopwords, not lemmatized

size = len(df) - 1
arr_q1_stem = X_q1_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_stem_d2v[i+1]
  arr_q1_stem = np.vstack([arr_q1_stem, arrT])

arr_q2_stem = X_q2_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_stem_d2v[i+1]
  arr_q2_stem = np.vstack([arr_q2_stem, arrT])

assert arr_q1_stem.shape == arr_q2_stem.shape

# arr_q1_stem = create_arrays(X_q1_stem_d2v)
# arr_q2_stem = create_arrays(X_q2_stem_d2v)

In [45]:
# Stemmed, with stopwords, not lemmatized 

size = len(df) - 1
arr_q1_sw_stem = X_q1_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_stem_d2v[i+1]
  arr_q1_sw_stem = np.vstack([arr_q1_sw_stem, arrT])

arr_q2_sw_stem = X_q2_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_stem_d2v[i+1]
  arr_q2_sw_stem = np.vstack([arr_q2_sw_stem, arrT])

assert arr_q1_sw_stem.shape == arr_q2_sw_stem.shape

# arr_q1_sw_stem = create_arrays(X_q1_sw_stem_d2v)
# arr_q2_sw_stem = create_arrays(X_q2_sw_stem_d2v)

In [46]:
# Without stopwords, not lemmatized, not stemmed
size = len(df) - 1
arr_q1_out = X_q1_out_d2v[0] 

for i in range(size):
  arrT = X_q1_out_d2v[i+1]
  arr_q1_out = np.vstack([arr_q1_out, arrT])

arr_q2_out = X_q2_out_d2v[0] 

for i in range(size):
  arrT = X_q2_out_d2v[i+1]
  arr_q2_out = np.vstack([arr_q2_out, arrT])

assert arr_q1_out.shape == arr_q2_out.shape

In [47]:
# With stopword, not lemmatized, not stemmed

size = len(df) - 1
arr_q1_sw = X_q1_sw_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_d2v[i+1]
  arr_q1_sw = np.vstack([arr_q1_sw, arrT])

arr_q2_sw = X_q2_sw_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_d2v[i+1]
  arr_q2_sw = np.vstack([arr_q2_sw, arrT])

assert arr_q1_sw.shape == arr_q2_sw.shape

### Convert to dataframe 

In [48]:
def df_arr(arr_q1, arr_q2, df):
  temp1 = pd.DataFrame(arr_q1)
  temp2 = pd.DataFrame(arr_q2)
  X_temp = pd.concat([temp1, temp2], axis=1)
  y = df['is_duplicate']
  y = np.array(y)
  y_df = pd.DataFrame(y)
  df_X = X_temp.join(y_df,  lsuffix='_left', rsuffix='_right')
  #X_temp['is_duplicate'] = df['is_duplicate']
  return df_X

In [49]:
# Dataframe - Lemmatized without stopwords 
lemma_df = df_arr(arr_q1_lemma, arr_q2_lemma, df)

In [50]:
# Dataframe - Lemmatized with stopwords 
sw_lemma_df = df_arr(arr_q1_sw_lemma, arr_q2_sw_lemma, df)

In [51]:
# Dataframe - Stemmed without stopwords 
stem_df = df_arr(arr_q1_stem, arr_q2_stem, df)

In [52]:
# Dataframe - Lemmatized with stopwords 
sw_stem_df = df_arr(arr_q1_sw_stem, arr_q2_sw_stem, df)

In [53]:
# Dataframe - With stopwords, not lemmatized, not stemmed
sw_df = df_arr(arr_q1_sw, arr_q2_sw, df)

In [54]:
# Dataframe - Without stopwords, not lemmatized, not stemmed
out_df = df_arr(arr_q1_out, arr_q2_out, df)

### Split Data

In [55]:
# Lemmatized with stopwords 
y_sw_lemma= sw_lemma_df['0_right']
X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma = train_test_split(sw_lemma_df, y_sw_lemma, stratify = y_sw_lemma, test_size = 0.15, random_state = 42)

In [56]:
# Lemmatized without stopwords 
y_lemma= lemma_df['0_right']
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(lemma_df, y_lemma, stratify = y_lemma, test_size = 0.15, random_state = 42)

In [57]:
# Stemmed with stopwords 
y_sw_stem= sw_stem_df['0_right']
X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem = train_test_split(sw_stem_df, y_sw_stem, stratify = y_sw_stem, test_size = 0.15, random_state = 42)

In [58]:
# Stemmed without stopwords 
y_stem= stem_df['0_right']
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(stem_df, y_stem, stratify = y_stem, test_size = 0.15, random_state = 42)

In [59]:
# With Stopwords, not lemmatized, not stemmed 
y_sw= sw_df['0_right']
X_train_sw, X_test_sw, y_train_sw, y_test_sw = train_test_split(sw_df, y_sw, stratify = y_sw, test_size = 0.15, random_state = 42)

In [60]:
# Without Stopwords, not lemmatized, not stemmed 
y_out= out_df['0_right']
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(out_df, y_out, stratify = y_out, test_size = 0.15, random_state = 42)

# MODELS

In [ ]:
# Creating DataFrame for Evaluations
result_se = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_se.head()

,Model Name,Accuracy,Precision,Recall,F1_score,Log_loss


In [ ]:
def res(model, acc, precision, recall, f1_score, log_loss):
    result_se.loc[len(result_se)]   = [model, acc, precision, recall, f1_score, log_loss]
    pd.set_option('display.max_rows', None)
    display (result_se)
    return result_se

### Random Forest

In [61]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    #splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)

    rf_clf = RandomForestClassifier(random_state = 0)
    rf_param_grid = { "max_depth": [3, None],
               "min_samples_split": [40]
              }
    
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, scoring='accuracy', resource='n_estimators', max_resources=5,              
                              random_state=0).fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_

    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model

In [ ]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred_rf, berf = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

RandomForestClassifier(min_samples_split=40, n_estimators=3, random_state=0)


In [ ]:
report_DV_rf = classification_report(y_pred_rf, y_test_lemma)
matrix_DV_rf = confusion_matrix(y_pred_rf, y_test_lemma)
print('Classification Report - RF\n', report_DV_rf)
print()
print('Confusion Matrix - RF\n', matrix_DV_rf)

Classification Report - RF
               precision    recall  f1-score   support

           0       0.99      0.94      0.96       507
           1       0.89      0.98      0.93       243

    accuracy                           0.95       750
   macro avg       0.94      0.96      0.95       750
weighted avg       0.96      0.95      0.95       750


Confusion Matrix - RF
 [[478  29]
 [  6 237]]


## Logistic Regression

In [ ]:
def lr_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Logistic Regression Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    lr_param_grid = {'C':[0.001]}     
    lr_clf = LogisticRegression(C=0.001, random_state = 0)
   
    lr_search = HalvingGridSearchCV(lr_clf, lr_param_grid, scoring = 'accuracy')
    lr_clf.fit(X_train, y_train)
    
    lr_model = lr_search.best_estimator_
    
    y_pred = lr_clf.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    #print (lr_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, lr_model

In [ ]:
acc_lr, precision_lr, recall_lr, f1_score_lr, log_loss_lr, y_pred_lr, belr  = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

ValueError: ignored

# Support Vector Classification - SVC

In [62]:
def svc_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for XGB Classifier and model fit'''
   
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    
    #svc_param_grid = {'C': [0.1, 10, 100], 'gamma': [1, 0.1, 0.01], 'kernel': ['sigmoid', 'linear', 'poly', 'rbf']}
    #svc_param_grid = {'C': [0.1, 100], 'gamma': [1, 0.1, 0.01], 'kernel': ['sigmoid', 'poly', 'rbf']}
    svc_param_grid = {'C': [10], 
              'gamma': [6], 
               'kernel': [ 'sigmoid']}
    svc_clf = SVC()
   # gamma='scale'
    svc_search = HalvingGridSearchCV(svc_clf, svc_param_grid, factor = 2, scoring = 'accuracy')
    svc_search.fit(X_train, y_train)
    
    svc_model = svc_search.best_params_
    svc_model_est = svc_search.best_estimator_
    
    y_pred = svc_model_est.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average='macro')
    print (svc_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, svc_model_est  

In [ ]:
acc_svc, precision_svc, recall_svc, f1_score_svc, log_loss_svc, y_pred_svc, besvc = svc_clf(X_train_lemma.values,
                                                                                            X_test_lemma.values, y_train_lemma, y_test_lemma)

{'C': 1, 'kernel': 'sigmoid'}


In [ ]:
report_svc = classification_report(y_pred_svc, y_test_lemma)
matrix_svc = confusion_matrix(y_pred_svc, y_test_lemma)
print('Classification Report - SVC\n', report_svc)
print()
print('Confusion Matrix - SVC\n', matrix_svc)

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4753
           1       1.00      1.00      1.00      2747

    accuracy                           1.00      7500
   macro avg       1.00      1.00      1.00      7500
weighted avg       1.00      1.00      1.00      7500


Confusion Matrix - SVC
 [[4751    2]
 [   0 2747]]


# Activate algorithm functions

#### On Lemmatized data

In [63]:
# Lemmatized, without stopwords  

#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)

# #Logistic Regression
# acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
# print()


# report_lr_lemma = classification_report(y_pred_lr_lemma, y_test_lemma)
# matrix_lr_lemma = confusion_matrix(y_pred_lr_lemma, y_test_lemma)
# print('Classification Report - Logistic Regression\n', report_lr_lemma)
# print()
# print('Confusion Matrix - Logistic Regression\n', matrix_lr_lemma)

# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma, besvc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

RandomForestClassifier(min_samples_split=40, n_estimators=5, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.97      0.94      0.96       485
           1       0.90      0.95      0.92       265

    accuracy                           0.95       750
   macro avg       0.94      0.95      0.94       750
weighted avg       0.95      0.95      0.95       750


Confusion Matrix - RF
 [[457  28]
 [ 13 252]]
{'C': 10, 'gamma': 6, 'kernel': 'sigmoid'}
Classification Report - SVC
               precision    recall  f1-score   support

           0       0.78      0.76      0.77       485
           1       0.58      0.61      0.59       265

    accuracy                           0.71       750
   macro avg       0.68      0.68      0.68       750
weighted avg       0.71      0.71      0.71       750


Confusion Matrix - SVC
 [[367 118]
 [103 162]]


In [ ]:
# Creating DataFrame for Evaluations
result_lemma = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_lemma.head()

,Model Name,Accuracy,Precision,Recall,F1_score,Log_loss


In [ ]:
# result_lemma = res('RandomForest Classifier',acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma)
# result_lemma = res('Logistic Regression', acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma)
# result_lemma = res('SVC Classifier', acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma, besvc_lemma)

## Lemmatized with stopwords

In [64]:
# Lemmatized, with stopwords

#Random Forest
acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma  = rf_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_rf_sw_lemma = classification_report(y_pred_rf_sw_lemma, y_test_sw_lemma)
matrix_rf_sw_lemma = confusion_matrix(y_pred_rf_sw_lemma, y_test_sw_lemma)
print('Classification Report - RF\n', report_rf_sw_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_lemma)
print()

# #Logistic Regression
# acc_lr_sw_lemma, precision_lr_sw_lemma, recall_lr_sw_lemma, f1_score_lr_sw_lemma, log_loss_lr_sw_lemma,  y_pred_lr_sw_lemma, belr_sw_lemma = lr_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)
# print()


# report_lr_sw_lemma = classification_report(y_pred_lr_sw_lemma, y_test_sw_lemma)
# matrix_lr_sw_lemma = confusion_matrix(y_pred_lr_sw_lemma, y_test_sw_lemma)
# print('Classification Report - Logistic Regression\n', report_lr_sw_lemma)
# print()
# print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw_lemma)


# SVC 
acc_svc_sw_lemma, precision_svc_sw_lemma, recall_svc_sw_lemma, f1_score_svc_sw_lemma, log_loss_svc_sw_lemma, y_pred_svc_sw_lemma, besvc_sw_lemma = svc_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_svc_sw_lemma = classification_report(y_pred_svc_sw_lemma, y_test_sw_lemma)
matrix_svc_sw_lemma = confusion_matrix(y_pred_svc_sw_lemma, y_test_sw_lemma)

print('Classification Report - SVC\n', report_svc_sw_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_lemma)

RandomForestClassifier(min_samples_split=40, n_estimators=5, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.97      0.92      0.95       495
           1       0.86      0.95      0.90       255

    accuracy                           0.93       750
   macro avg       0.92      0.93      0.92       750
weighted avg       0.93      0.93      0.93       750


Confusion Matrix - RF
 [[456  39]
 [ 14 241]]

{'C': 10, 'gamma': 6, 'kernel': 'sigmoid'}
Classification Report - SVC
               precision    recall  f1-score   support

           0       0.72      0.71      0.72       472
           1       0.52      0.52      0.52       278

    accuracy                           0.64       750
   macro avg       0.62      0.62      0.62       750
weighted avg       0.64      0.64      0.64       750


Confusion Matrix - SVC
 [[337 135]
 [133 145]]


In [ ]:
# Creating DataFrame for Evaluations
result_sw_lemma = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_sw_lemma.head()

In [ ]:
# result_sw_lemma = res('RandomForest Classifier', acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma)
# result_sw_lemma = res('Logistic Regression', acc_lr_sw_lemma, precision_lr_sw_lemma, recall_lr_sw_lemma, f1_score_lr_sw_lemma, log_loss_lr_sw_lemma,  y_pred_lr_sw_lemma, belr_sw_lemma)
# result_sw_lemma = res('SVC Classifier', acc_svc_sw_lemma, precision_svc_sw_lemma, recall_svc_sw_lemma, f1_score_svc_sw_lemma, log_loss_svc_sw_lemma, y_pred_svc_sw_lemma, besvc_sw_lemma)

# With stopwords, not lemmatized nor stemmed

In [65]:
# With stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw  = rf_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_rf_sw = classification_report(y_pred_rf_sw, y_test_sw)
matrix_rf_sw = confusion_matrix(y_pred_rf_sw, y_test_sw)
print('Classification Report - RF\n', report_rf_sw)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw)
print()

# #Logistic Regression
# acc_lr_sw, precision_lr_sw, recall_lr_sw, f1_score_lr_sw, log_loss_lr_sw,  y_pred_lr_sw, belr_sw = lr_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)
# print()
# report_lr_sw = classification_report(y_pred_lr_sw, y_test_sw)
# matrix_lr_sw = confusion_matrix(y_pred_lr_sw, y_test_sw)
# print('Classification Report - Logistic Regression\n', report_lr_sw)
# print()
# print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw)


# SVC 
acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw, besvc_sw= svc_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_svc_sw = classification_report(y_pred_svc_sw, y_test_sw)
matrix_svc_sw = confusion_matrix(y_pred_svc_sw, y_test_sw)

print('Classification Report - SVC\n', report_svc_sw)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw)

RandomForestClassifier(min_samples_split=40, n_estimators=5, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.97      0.94      0.95       483
           1       0.90      0.94      0.92       267

    accuracy                           0.94       750
   macro avg       0.93      0.94      0.94       750
weighted avg       0.94      0.94      0.94       750


Confusion Matrix - RF
 [[455  28]
 [ 15 252]]

{'C': 10, 'gamma': 6, 'kernel': 'sigmoid'}
Classification Report - SVC
               precision    recall  f1-score   support

           0       0.69      0.69      0.69       470
           1       0.49      0.49      0.49       280

    accuracy                           0.62       750
   macro avg       0.59      0.59      0.59       750
weighted avg       0.62      0.62      0.62       750


Confusion Matrix - SVC
 [[326 144]
 [144 136]]


In [ ]:
# Creating DataFrame for Evaluations
result_sw = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_sw.head()

In [ ]:
# result_sw = res('RandomForest Classifier', acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw)
# result_sw = res('Logistic Regression',acc_lr_sw, precision_lr_sw, recall_lr_sw, f1_score_lr_sw, log_loss_lr_sw,  y_pred_lr_sw, belr_sw)
# result_sw = res('SVC Classifier', acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw, besvc_sw)

### Without stopwords, not lemmatized nor stemmed

In [66]:
# Without stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_out, precision_rf_out, recall_rf_out, f1_score_rf_out, log_loss_rf_out, y_pred_rf_out, berf_out  = rf_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_rf_out = classification_report(y_pred_rf_out, y_test_out)
matrix_rf_out = confusion_matrix(y_pred_rf_out, y_test_out)
print('Classification Report - RF\n', report_rf_out)
print()
print('Confusion Matrix - RF\n', matrix_rf_out)
print()

# #Logistic Regression
# acc_lr_out, precision_lr_out, recall_lr_out, f1_score_lr_out, log_loss_lr_out,  y_pred_lr_out, belr_out = lr_clf(X_train_out, X_test_out, y_train_out, y_test_out)
# print()
# report_lr_out = classification_report(y_pred_lr_out, y_test_out)
# matrix_lr_out = confusion_matrix(y_pred_lr_out, y_test_out)
# print('Classification Report - Logistic Regression\n', report_lr_out)
# print()
# print('Confusion Matrix - Logistic Regression\n', matrix_lr_out)


# SVC 
acc_svc_out, precision_svc_out, recall_svc_out, f1_score_svc_out, log_loss_svc_out, y_pred_svc_out, besvc_out = svc_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_svc_out = classification_report(y_pred_svc_out, y_test_out)
matrix_svc_out = confusion_matrix(y_pred_svc_out, y_test_out)

print('Classification Report - SVC\n', report_svc_out)
print()
print('Confusion Matrix - SVC\n', matrix_svc_out)

RandomForestClassifier(min_samples_split=40, n_estimators=5, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       471
           1       0.95      0.96      0.96       279

    accuracy                           0.97       750
   macro avg       0.96      0.96      0.96       750
weighted avg       0.97      0.97      0.97       750


Confusion Matrix - RF
 [[458  13]
 [ 12 267]]

{'C': 10, 'gamma': 6, 'kernel': 'sigmoid'}
Classification Report - SVC
               precision    recall  f1-score   support

           0       0.79      0.78      0.78       474
           1       0.62      0.63      0.63       276

    accuracy                           0.73       750
   macro avg       0.71      0.71      0.71       750
weighted avg       0.73      0.73      0.73       750


Confusion Matrix - SVC
 [[369 105]
 [101 175]]


In [ ]:
# Creating DataFrame for Evaluations
result_out = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_out.head()

In [ ]:
# result_out = res('RandomForest Classifier', acc_rf_out, precision_rf_out, recall_rf_out, f1_score_rf_out, log_loss_rf_out, y_pred_rf_out, berf_out)
# result_out = res('Logistic Regression', acc_lr_out, precision_lr_out, recall_lr_out, f1_score_lr_out, log_loss_lr_out,  y_pred_lr_out, belr_out)
# result_out = res('SVC Classifier', acc_svc_out, precision_svc_out, recall_svc_out, f1_score_svc_out, log_loss_svc_out, y_pred_svc_out, besvc_out)

# Stemmed data with stopwords

In [ ]:
# Stemmed with stopwords

#Random Forest
acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem  = rf_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_rf_sw_stem = classification_report(y_pred_rf_sw_stem, y_test_sw_stem)
matrix_rf_sw_stem = confusion_matrix(y_pred_rf_sw_stem, y_test_sw_stem)
print('Classification Report - RF\n', report_rf_sw_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_stem)
print()


# #Logistic Regression
# acc_lr_sw_stem, precision_lr_sw_stem, recall_lr_sw_stem, f1_score_lr_sw_stem, log_loss_lr_sw_stem,  y_pred_lr_sw_stem, belr_sw_stem = lr_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)
# print()
# report_lr_sw_stem = classification_report(y_pred_lr_sw_stem, y_test_sw_stem)
# matrix_lr_sw_stem = confusion_matrix(y_pred_lr_sw_stem, y_test_sw_stem)
# print('Classification Report - Logistic Regression\n', report_lr_sw_stem)
# print()
# print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw_stem)


# SVC 
acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem, besvc_sw_stem = svc_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_svc_sw_stem = classification_report(y_pred_svc_sw_stem, y_test_sw_stem)
matrix_svc_sw_stem = confusion_matrix(y_pred_svc_sw_stem, y_test_sw_stem)

print('Classification Report - SVC\n', report_svc_sw_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_stem)

RandomForestClassifier(min_samples_split=40, n_estimators=5, random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       486
           1       0.91      0.97      0.94       264

    accuracy                           0.96       750
   macro avg       0.95      0.96      0.95       750
weighted avg       0.96      0.96      0.96       750


Confusion Matrix - RF
 [[462  24]
 [  8 256]]



In [ ]:
# Creating DataFrame for Evaluations
result_sw_stem = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_sw_stem.head()

In [ ]:
# result_sw_stem = res('RandomForest Classifier', acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem)
# result_sw_stem = res('Logistic Regression', acc_lr_sw_stem, precision_lr_sw_stem, recall_lr_sw_stem, f1_score_lr_sw_stem, log_loss_lr_sw_stem,  y_pred_lr_sw_stem, belr_sw_stem)
# result_sw_stem = res('SVC Classifier', acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem, besvc_sw_stem)

# Stemmed without stopwords


In [ ]:
# Stemmed with stopwords

#Random Forest
acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem  = rf_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_rf_stem = classification_report(y_pred_rf_stem, y_test_stem)
matrix_rf_stem = confusion_matrix(y_pred_rf_stem, y_test_stem)
print('Classification Report - RF\n', report_rf_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_stem)
print()

# #Logistic Regression
# acc_lr_stem, precision_lr_stem, recall_lr_stem, f1_score_lr_stem, log_loss_lr_stem,  y_pred_lr_stem, belr_stem = lr_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)
# print()
# report_lr_stem = classification_report(y_pred_lr_stem, y_test_stem)
# matrix_lr_stem = confusion_matrix(y_pred_lr_stem, y_test_stem)
# print('Classification Report - Logistic Regression\n', report_lr_stem)
# print()
# print('Confusion Matrix - Logistic Regression\n', matrix_lr_stem)

# SVC 
acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem, besvc_stem = svc_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_svc_stem = classification_report(y_pred_svc_stem, y_test_stem)
matrix_svc_stem = confusion_matrix(y_pred_svc_stem, y_test_stem)

print('Classification Report - SVC\n', report_svc_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_stem)

In [ ]:
# Creating DataFrame for Evaluations
result_stem = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_stem.head()

In [ ]:
# result_stem = res('RandomForest Classifier', acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem)
# result_stem = res('Logistic Regression', acc_lr_stem, precision_lr_stem, recall_lr_stem, f1_score_lr_stem, log_loss_lr_stem,  y_pred_lr_stem, belr_stem)
# result_stem = res('SVC Classifier', acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem, besvc_stem)

# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#cosine = cosine_similarity(X_train_lemma, y_train_lemma, dense_output=True)

In [ ]:
cosine_similarity(arr_q1_lemma , arr_q2_lemma, dense_output=True)

array([[0.77353704, 0.6493094 , 0.7844405 , ..., 0.61083734, 0.6324987 ,
        0.8206404 ],
       [0.33382827, 0.8227184 , 0.5351572 , ..., 0.5696451 , 0.4045978 ,
        0.5432366 ],
       [0.57329106, 0.65007234, 0.86115694, ..., 0.77194357, 0.52491057,
        0.7496874 ],
       ...,
       [0.5479884 , 0.5654626 , 0.74211514, ..., 0.8167627 , 0.58999306,
        0.8312438 ],
       [0.63509995, 0.54801077, 0.78231716, ..., 0.63727254, 0.88014674,
        0.81301326],
       [0.6825827 , 0.6387254 , 0.84284997, ..., 0.6468858 , 0.6542733 ,
        0.8440794 ]], dtype=float32)

In [ ]:
cosine_similarity(arr_q1_lemma , arr_q2_lemma)
cosine_similarity(arr_q1_sw_lemma , arr_q2_sw_lemma)
cosine_similarity(arr_q1_stem , arr_q2_stem)
cosine_similarity(arr_q1_sw_stem , arr_q2_sw_stem)
cosine_similarity(arr_q1_sw , arr_q2_sw)
cosine_similarity(arr_q1_out , arr_q2_out)

# Prediction

In [ ]:
def prediction(s1, s2, model):
    import pandas as pd
    import numpy as np
    import re
    import string
    from string import punctuation
    import nltk
    from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
    from nltk.corpus import wordnet, stopwords
    from nltk import pos_tag, word_tokenize
    import gensim
    from gensim.parsing.preprocessing import remove_stopwords
    from gensim.models.doc2vec import Doc2Vec, TaggedDocument
    from keras.preprocessing.text import Tokenizer
    import warnings
    warnings.filterwarnings('ignore')
    
    stops = set(stopwords.words('english'))
    
    stemmer =  PorterStemmer()
    lemmer = WordNetLemmatizer()

    def get_wordnet_pos(tag):
        if tag.startswith("N"):
            return wordnet.NOUN
        if tag.startswith("J"):
            return wordnet.ADJ
        if tag.startswith("V"):
            return wordnet.VERB
        if tag.startswith("R"):
            return wordnet.ADV
        return wordnet.NOUN

    def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
        sentence = sentence.lower().strip()
        sentence = sentence.encode("ascii", errors="ignore").decode()
        sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
   
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
        if stopwords:
            sentence = remove_stopwords(sentence) 
    
        not_1 = []
        for word in sentence.split():
            if len(word) > 1:
                not_1.append(word)
        sentence = ' '.join(not_1) 

        if lemmatize:
            nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
            wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
            res_words = []
            for word, tag in wn_tagged:
                if tag is None:
                    res_words.append(word)
                else :
                    res_words.append(lemmer.lemmatize(word, tag))
            sentence = " ".join(res_words)
        
        if stem:
            sent_stemmed = ''
            for word in sentence.split():
                sent_stemmed += ' '+ stemmer.stem(word)
            sentence = sent_stemmed
    
        return sentence

    def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
        cleaned_sentences1 = []
        cleaned_sentences2 = []
    
        for index, row in df.iterrows():
       
            cleaned1 = clean_sentence(row['qs1'], stopwords, lemmatize, stem) 
            cleaned2 = clean_sentence(row['qs2'], stopwords, lemmatize, stem)

            cleaned_sentences1.append(cleaned1)
            cleaned_sentences2.append(cleaned2)
        return cleaned_sentences1, cleaned_sentences2

    def to_df(q1cleaned, q2cleaned, df):
        '''cleaned sentences to df'''
        X_temp1 = pd.DataFrame()
        X_temp2 = pd.DataFrame()
        X_temp1['qs1'] = pd.DataFrame(q1cleaned, index = df.index)
        X_temp2['qs2'] = pd.DataFrame(q2cleaned, index = df.index)
        X_temp = pd.concat([X_temp1, X_temp2], axis = 1) 
        return X_temp 

    sdf = pd.DataFrame(columns = ['qs1', 'qs2'])
    sdf.loc[len(sdf)]   = [s1, s2]
    
    q1_swt, q2_swt = get_cleaned_senteces(sdf, stopwords = False, lemmatize = True, stem = False)
    assert len(q1_swt) == len(q2_swt)
  
    s1 = [TaggedDocument(d, [i]) for i, d in enumerate(q1_swt)]
    s2 = [TaggedDocument(d, [i]) for i, d in enumerate(q2_swt)]

    cv1 = Doc2Vec(s1, dm = 1, vector_size = 100, window = 3, min_count = 2, epochs = 30)
    cv2 = Doc2Vec(s2, dm = 1, vector_size = 100, window = 3, min_count = 2, epochs = 30)

    arr_s1 = np.array(cv1[0]).reshape(1, 100)
    arr_s2 = np.array(cv2[0]).reshape(1, 100)
    temp1 = pd.DataFrame(arr_s1)
    temp2 = pd.DataFrame(arr_s2)
    X_temp = pd.concat([temp1, temp2], axis=1)
    X_temp['y'] = 5
  
    predict = model.predict(X_temp)
    return predict


In [ ]:
prediction(s1, s2, besvc)

array([1])

In [ ]:
# label = 0
s1 = "How many keywords are there in PERL Programming Language in the latest version?"
s2 = "How many keywords are there in the Racket programming language of the latest version?"
s1, s2 = s2, s1

In [ ]:
# Label 1
s3 = "What should I do to be a great geologist?"
s4 = "How can I be a great geologist?"
s1, s2 = s1, s2

In [ ]:
# 0, 1
print('Prediction Random Forest', prediction(s1, s2, berf), prediction(s3, s4, berf))
#print('Prediction Logistic Regression', prediction(s1, s2, belr), prediction(s3, s4, belr))
#print('Prediction XGB', prediction(s2, s1, bexgb), prediction(s3, s4, bexgb))
print('Prediction SVC', prediction(s1, s2, besvc), prediction(s3, s4, besvc))
#print('Prediction Linear SVC', prediction(s1, s2, belsvc), prediction(s3, s4, belsvc))
#print('Prediction LighGBM', prediction(s1, s2, belgbm), prediction(s3, s4, belgbm))
#print('Prediction SGD', prediction(s1, s2, besgd), prediction(s3, s4, besgd))
#print('Prediction modelTf', prediction(s1, s2, modelTf), prediction(s3, s4, modelTf))

Prediction Random Forest [0] [0]


Prediction SVC [1] [1]
